In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip -q gdrive/MyDrive/plants/models/model_mobilenetv2_regularized_grasp_125.zip
!unzip -q gdrive/MyDrive/plants/models/model_squeeze_excite_resnet_grasp_125.zip
!unzip -q gdrive/MyDrive/plants/models/model_pca_based_grasp_125.zip
!unzip -q gdrive/MyDrive/plants/models/model_pca_sepconv_grasp_125.zip
!unzip -q gdrive/MyDrive/plants/models/model_lda_basd_grasp_125.zip
!unzip -q gdrive/MyDrive/plants/models/model_lda_sepconv_grasp_125.zip

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import pandas as pd
from matplotlib.patches import Patch
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from tensorflow import keras

In [ ]:
model_mn2 = keras.applications.MobileNetV2(
    input_shape=None, alpha=1.0, include_top=True, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000
)
layer_name = 'block_15_add'
feature_extractor = keras.models.Model(inputs=model_mn2.input, outputs=model_mn2.get_layer(layer_name).output)

In [ ]:
feature_extractor.summary()

In [ ]:
model_mobilenetv2_regularized = tf.keras.models.load_model("model_mobilenetv2_regularized")
model_squeeze_excite_resnet = tf.keras.models.load_model("model_squeeze_excite_resnet")
model_pca_based = tf.keras.models.load_model("model_pca_based")
model_pca_sepconv = tf.keras.models.load_model("model_pca_sepconv")
model_lda_based = tf.keras.models.load_model("model_lda_based")
model_lda_sepconv = tf.keras.models.load_model("model_lda_sepconv")

In [ ]:
model_mobilenetv2_regularized.summary()

In [ ]:
model_squeeze_excite_resnet.summary()

In [ ]:
model_pca_based.summary()

In [ ]:
model_pca_sepconv.summary()

In [ ]:
model_lda_based.summary()

In [ ]:
model_lda_sepconv.summary()

In [ ]:
plants_names = []

for plant in os.listdir('gdrive/MyDrive/plants/grasp_125/encoded/train'):
    plants_names += [plant.split('.npy')[0]]

plants_names.sort()
plant_name_to_id = {plant: i for i, plant in enumerate(plants_names)}

In [ ]:
instances = {}
labels = {}

for set_kind in ["test", "train", "val"]:
    X_per_class = []
    y_per_class = []

    for i, file in enumerate(os.listdir(f"gdrive/MyDrive/plants/grasp_125/encoded/{set_kind}")):
        print(f"\r[{i}] {set_kind}/{file}", end='')
        plants_encoded = np.load(f"gdrive/MyDrive/plants/grasp_125/encoded/{set_kind}/{file}")
        plant_name = file.split('.npy')[0]
        
        X_per_class += [plants_encoded.reshape([plants_encoded.shape[0], 49, 160])]
        y_per_class += [plant_name_to_id[plant_name] for _ in range(plants_encoded.shape[0])]

    instances[set_kind] = np.concatenate(X_per_class)
    labels[set_kind] = np.array(y_per_class)

[124] val/894.npy

In [ ]:
n_classes = len(plants_names)

X_train = instances['train']
y_train = np.eye(n_classes)[labels['train']]

X_valid = instances['val']
y_valid = np.eye(n_classes)[labels['val']]

X_test = instances['test']
y_test = np.eye(n_classes)[labels['test']]

In [ ]:
grasp_125_classes = pd.read_csv("gdrive/MyDrive/plants/grasp-125_classes.csv", header=None)

In [ ]:
def get_name_from_id(number):
  number = int(number)
  return list(grasp_125_classes[grasp_125_classes[0] == number][1])[0]

In [ ]:
model_mnv2 = keras.models.Sequential([
    keras.layers.Input(shape=(224, 224, 3), dtype=tf.uint8),
    keras.layers.Lambda(lambda x: tf.cast(x, dtype=tf.float32)),
    keras.layers.Lambda(
        tf.keras.applications.mobilenet_v2.preprocess_input, 
        name='preprocessing',
        input_shape=(224, 224, 3),
        dtype=tf.float32),
    feature_extractor,
    model_mobilenetv2_regularized
])

In [ ]:
model_seresnet = keras.models.Sequential([
    keras.layers.Input(shape=(224, 224, 3), dtype=tf.uint8),
    keras.layers.Lambda(lambda x: tf.cast(x, dtype=tf.float32)),
    keras.layers.Lambda(
        tf.keras.applications.mobilenet_v2.preprocess_input, 
        name='preprocessing',
        input_shape=(224, 224, 3),
        dtype=tf.float32),
    feature_extractor,
    model_squeeze_excite_resnet
])

In [ ]:
pca_per_filter = [
    PCA(n_components=5).fit(X_train[..., i])
    for i in range(X_train.shape[-1])
]

class PCATransformer(keras.layers.Layer):
    def __init__(self, means, components, **kwargs):
        super().__init__(**kwargs)
        self.all_means = tf.reshape(tf.transpose(means), [1, 49, 160])
        self.all_components = tf.transpose(components, perm=[0, 2, 1])
    
    @tf.function
    def call(self, X):
        return tf.transpose(tf.map_fn(
            lambda inp: inp[0] @ inp[1],
            (tf.transpose(X - self.all_means, perm=[2, 0, 1]), self.all_components),
            fn_output_signature=tf.TensorSpec(shape=[None, 5])
        ), perm=[1, 2, 0])
    
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape((batch_input_shape[0], 5, 160))
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "mean": self.means.numpy(), "components": self.components.numpy()}


all_means = np.array([filter.mean_ for filter in pca_per_filter])
all_components = np.array([filter.components_ for filter in pca_per_filter])

model_pcadense = keras.models.Sequential([
    keras.layers.Input(shape=(224, 224, 3), dtype=tf.uint8),
    keras.layers.Lambda(lambda x: tf.cast(x, dtype=tf.float32)),
    keras.layers.Lambda(
        tf.keras.applications.mobilenet_v2.preprocess_input, 
        name='preprocessing',
        input_shape=(224, 224, 3),
        dtype=tf.float32),
    feature_extractor,
    keras.layers.Reshape(target_shape=[49, 160]),
    PCATransformer(tf.constant(all_means), tf.constant(all_components)),
    keras.layers.Flatten(input_shape=[5, 160]),
    model_pca_based
])

model_pcasepconv = keras.models.Sequential([
    keras.layers.Input(shape=(224, 224, 3), dtype=tf.uint8),
    keras.layers.Lambda(lambda x: tf.cast(x, dtype=tf.float32)),
    keras.layers.Lambda(
        tf.keras.applications.mobilenet_v2.preprocess_input, 
        name='preprocessing',
        input_shape=(224, 224, 3),
        dtype=tf.float32),
    feature_extractor,
    keras.layers.Reshape(target_shape=[49, 160]),
    PCATransformer(tf.constant(all_means), tf.constant(all_components)),
    model_pca_sepconv
])

In [ ]:
pca_transformation = PCATransformer(tf.constant(all_means), tf.constant(all_components))

In [ ]:
lda_per_filter = [
    LDA(n_components=5).fit(X_train[..., i], np.argmax(y_train, axis=1))
    for i in range(X_train.shape[-1])
]

class LDATransformer(keras.layers.Layer):
    def __init__(self, means, components, **kwargs):
        super().__init__(**kwargs)
        self.all_means = tf.reshape(tf.transpose(means), [1, 49, 160])
        self.all_components = tf.transpose(components, perm=[0, 1, 2])
    
    @tf.function
    def call(self, X):
        return tf.transpose(tf.map_fn(
            lambda inp: inp[0] @ inp[1],
            (tf.transpose(X - self.all_means, perm=[2, 0, 1]), self.all_components),
            fn_output_signature=tf.TensorSpec(shape=[None, 5])
        ), perm=[1, 2, 0])
    
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape((batch_input_shape[0], 5, 160))
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "mean": self.means.numpy(), "components": self.components.numpy()}


all_means = np.array([filter.xbar_ for filter in lda_per_filter], dtype='float32')
all_components = np.array([filter.scalings_[:, :5] for filter in lda_per_filter], dtype='float32')

In [ ]:
lda_transformation = LDATransformer(tf.constant(all_means), tf.constant(all_components))

In [ ]:
def top_k_accuracy_score(y_true, y_pred, k=1):
    ranks = np.sum(y_pred >= y_pred[y_true == 1].reshape(-1, 1), axis=1)
    return np.sum(ranks <= k) / ranks.shape[0]

In [ ]:
print("Validation set accuracy")
preds_valid_1 = model_mobilenetv2_regularized(tf.reshape(X_valid, [-1, 7, 7, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_1, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_1, k=5))

print("Test set")
preds_test_aug_1 = model_mobilenetv2_regularized(tf.reshape(X_test, [-1, 7, 7, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_1, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_1, k=5))

In [ ]:
print("Validation set accuracy")
preds_valid_2 = model_squeeze_excite_resnet(tf.reshape(X_valid, [-1, 7, 7, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_2, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_2, k=5))

print("Test set")
preds_test_aug_2 = model_squeeze_excite_resnet(tf.reshape(X_test, [-1, 7, 7, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_2, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_2, k=5))

In [ ]:
print("Validation set accuracy")
preds_valid_3 = model_pca_based(tf.reshape(pca_transformation(X_valid), [-1, 800])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_3, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_3, k=5))

print("Test set")
preds_test_aug_3 = model_pca_based(tf.reshape(pca_transformation(X_test), [-1, 800])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_3, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_3, k=5))

In [ ]:
print("Validation set accuracy")
preds_valid_4 = model_pca_sepconv(tf.reshape(pca_transformation(X_valid), [-1, 5, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_4, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_4, k=5))

print("Test set")
preds_test_aug_4 = model_pca_sepconv(tf.reshape(pca_transformation(X_test), [-1, 5, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_4, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_4, k=5))

In [ ]:
print("Validation set accuracy")
preds_valid_4 = model_lda_based(tf.reshape(lda_transformation(X_valid), [-1, 800])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_4, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_4, k=5))

print("Test set")
preds_test_aug_4 = model_lda_based(tf.reshape(lda_transformation(X_test), [-1, 800])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_4, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_4, k=5))

In [ ]:
print("Validation set accuracy")
preds_valid_5 = model_lda_sepconv(tf.reshape(lda_transformation(X_valid), [-1, 5, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_5, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_5, k=5))

print("Test set")
preds_test_aug_4 = model_lda_sepconv(tf.reshape(lda_transformation(X_test), [-1, 5, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_4, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_4, k=5))